# Wind Downscaling

## Prerequisites

* Conda environment
* Get a Copernicus API key from: https://cds.climate.copernicus.eu/api-how-to
  * create a file at \$HOME/.cdsapirc with the required UID and key
* Create a .env file in the same folder as this notebook, and add the COSMO_USERNAME and COSMO_PASSWORD to connect to the UNI-BE server

## Install the required packages

In [1]:
from datetime import date
from pathlib import Path
import os

In [2]:
if Path('./setup.py').exists():
    !pip install -e .
else:
    !pip install -U git+https://github.com/OpheliaMiralles/WindDownscaling_EPFL_UNIBE.git

Obtaining file:///Users/Boubou/Documents/GitHub/WindDownscaling_EPFL_UNIBE
  Attempting uninstall: downscaling
    Found existing installation: downscaling 1.0
    Uninstalling downscaling-1.0:
      Successfully uninstalled downscaling-1.0
  Running setup.py develop for downscaling


In [3]:
!conda install -y -c conda-forge gdal tensorflow xarray numpy=1.19.5 pandas pysftp cdsapi elevation rasterio dask python-dotenv

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.1
  latest version: 4.10.3

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.



In [4]:
!pip install topo-descriptors

In [5]:
from dotenv import load_dotenv
load_dotenv()
import numpy as np
import tensorflow as tf
import tensorflow.keras.callbacks as cb
print(f"Num GPUs Available: {len(tf.config.list_physical_devices('GPU'))}")

Num GPUs Available: 0


## Set configuration

In [6]:
DATA_ROOT = Path('./data')
ERA5_DATA_FOLDER = DATA_ROOT / 'ERA5'
COSMO1_DATA_FOLDER = DATA_ROOT / 'COSMO1'
DEM_DATA_FILE = DATA_ROOT / 'dem/Switzerland-90m-DEM.tif'
PROCESSED_DATA_FOLDER = DATA_ROOT / 'img_prediction_files'

DATA_ROOT.mkdir(parents=True, exist_ok=True)
ERA5_DATA_FOLDER.mkdir(exist_ok=True)
COSMO1_DATA_FOLDER.mkdir(exist_ok=True)
DEM_DATA_FILE.parent.mkdir(exist_ok=True)
PROCESSED_DATA_FOLDER.mkdir(exist_ok=True)

In [7]:
ERA5_PREDICTORS_SURFACE = ('u10', 'v10', 'blh', 'fsr', 'sp', 'sshf',
                               'u100', 'v100')
ERA5_PREDICTORS_Z500 = ('d', 'z', 'u', 'v', 'w', 'vo')
TOPO_PREDICTORS = ('tpi_500', 'ridge_index_norm', 'ridge_index_dir',
                   'we_derivative', 'sn_derivative',
                   'slope', 'aspect')
ALL_INPUTS = ['u10', 'sp']+ list(TOPO_PREDICTORS) #ERA5_PREDICTORS_SURFACE + TOPO_PREDICTORS
ALL_OUTPUTS = ['U_10M']

In [8]:
# Start and end date for the data - should be in the 2016-2020 range
START_DATE = date(2016,4,1)
END_DATE = date(2016,6,1)
NUM_DAYS = (END_DATE-START_DATE).days + 1
# Number of consecutive images to form a sequence
SEQUENCE_LENGTH = 3
# Size of the high resolution image to be produced
IMG_SIZE = 128
# Number of noise channels to add to the image
NOISE_CHANNELS = 1000
# Number of sequences per batch
BATCH_SIZE = 8
# Number of workers to run to process the data to create the batches
BATCH_WORKERS = 8
# Latent dimension for the autoencoder
USE_AUTOENCODER = True
AUTOENCODER_OUTPUT_FEATURES = 8

## Data Loading

In [9]:
from data.download_ERA5 import download_ERA5
download_ERA5(ERA5_DATA_FOLDER, START_DATE, END_DATE)
print('Done')

File 20160401_era5_surface_hourly already exists
File 20160402_era5_surface_hourly already exists
File 20160403_era5_surface_hourly already exists
File 20160404_era5_surface_hourly already exists
File 20160405_era5_surface_hourly already exists
File 20160406_era5_surface_hourly already exists
File 20160407_era5_surface_hourly already exists
File 20160408_era5_surface_hourly already exists
File 20160409_era5_surface_hourly already exists
File 20160410_era5_surface_hourly already exists
File 20160411_era5_surface_hourly already exists
File 20160412_era5_surface_hourly already exists
File 20160413_era5_surface_hourly already exists
File 20160414_era5_surface_hourly already exists
File 20160415_era5_surface_hourly already exists
File 20160416_era5_surface_hourly already exists
File 20160417_era5_surface_hourly already exists
File 20160418_era5_surface_hourly already exists
File 20160419_era5_surface_hourly already exists
File 20160420_era5_surface_hourly already exists
File 20160421_era5_s

In [10]:
if not DEM_DATA_FILE.exists():
    dest = str(DEM_DATA_FILE)
    !eio --product SRTM3 clip -o {dest} --bounds 5.27 45.46 11.02 48.15

In [11]:
from data.data_processing import process_topographic_variables_file
process_topographic_variables_file(DEM_DATA_FILE)

2021-09-21 15:20:41,221 INFO Loading /Users/Boubou/opt/anaconda3/envs/downscale_dev/lib/python3.9/site-packages/topo_descriptors/config/topo_descriptors.conf.
2021-09-21 15:20:41,222 INFO Loading configuration file: /Users/Boubou/opt/anaconda3/envs/downscale_dev/lib/python3.9/site-packages/topo_descriptors/config/topo_descriptors.conf


Already processed all topo files


In [12]:
username = os.getenv('COSMO_USERNAME')
password = os.getenv('COSMO_PASSWORD')
from data import download_COSMO1
download_COSMO1(username, password, COSMO1_DATA_FOLDER, START_DATE, END_DATE)

Finished downloading COSMO data


In [13]:
from data.data_processing import process_imgs
process_imgs(PROCESSED_DATA_FOLDER, ERA5_DATA_FOLDER, COSMO1_DATA_FOLDER, DEM_DATA_FILE.parent,
             surface_variables_included=ERA5_PREDICTORS_SURFACE,
             z500_variables_included=ERA5_PREDICTORS_Z500,
             topo_variables_included=TOPO_PREDICTORS,
             cosmo_variables_included=('U_10M', 'V_10M'),
             start_date=START_DATE, end_date=END_DATE)
print('Done')

Reading DEM data files
Inputs and outputs for date 20160401 have already been processed.
Inputs and outputs for date 20160402 have already been processed.
Inputs and outputs for date 20160403 have already been processed.
Inputs and outputs for date 20160404 have already been processed.
Inputs and outputs for date 20160405 have already been processed.
Inputs and outputs for date 20160406 have already been processed.
Inputs and outputs for date 20160407 have already been processed.
Inputs and outputs for date 20160408 have already been processed.
Inputs and outputs for date 20160409 have already been processed.
Inputs and outputs for date 20160410 have already been processed.
Inputs and outputs for date 20160411 have already been processed.
Inputs and outputs for date 20160412 have already been processed.
Inputs and outputs for date 20160413 have already been processed.
Inputs and outputs for date 20160414 have already been processed.
Inputs and outputs for date 20160415 have already bee

In [14]:
from data.data_generator import BatchGenerator, NaiveDecoder

batch_gen = BatchGenerator(path_to_data=PROCESSED_DATA_FOLDER, decoder=NaiveDecoder(normalize=True),
                           sequence_length=SEQUENCE_LENGTH,
                           patch_length_pixel=IMG_SIZE, batch_size=BATCH_SIZE,
                           input_variables=ALL_INPUTS,
                           output_variables= ALL_OUTPUTS,
                           start_date=START_DATE, end_date=END_DATE,
                           num_workers=BATCH_WORKERS)

inputs = []
outputs = []
with batch_gen as batch:
    for b in range(NUM_DAYS):
        print(f'Creating batch {b+1}/{NUM_DAYS}')
        x, y = next(batch)
        inputs.append(x)
        outputs.append(y)
inputs = np.concatenate(inputs, axis=0)
outputs = np.concatenate(outputs, axis=0)
print(f"Inputs: {inputs.shape}")
print(f"Outputs: {outputs.shape}")

Creating batch 1/62


2021-09-21 15:20:46,945 WARNING multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.


Creating batch 2/62
Creating batch 3/62
Creating batch 4/62
Creating batch 5/62
Creating batch 6/62
Creating batch 7/62
Creating batch 8/62
Creating batch 9/62
Creating batch 10/62
Creating batch 11/62
Creating batch 12/62
Creating batch 13/62
Creating batch 14/62
Creating batch 15/62
Creating batch 16/62
Creating batch 17/62
Creating batch 18/62
Creating batch 19/62
Creating batch 20/62
Creating batch 21/62
Creating batch 22/62
Creating batch 23/62
Creating batch 24/62
Creating batch 25/62
Creating batch 26/62
Creating batch 27/62
Creating batch 28/62
Creating batch 29/62
Creating batch 30/62
Creating batch 31/62
Creating batch 32/62
Creating batch 33/62
Creating batch 34/62
Creating batch 35/62
Creating batch 36/62
Creating batch 37/62
Creating batch 38/62
Creating batch 39/62
Creating batch 40/62
Creating batch 41/62
Creating batch 42/62
Creating batch 43/62
Creating batch 44/62
Creating batch 45/62
Creating batch 46/62
Creating batch 47/62
Creating batch 48/62
Creating batch 49/62


In [15]:
INPUT_CHANNELS = len(ALL_INPUTS)
OUT_CHANNELS = len(ALL_OUTPUTS)

if USE_AUTOENCODER:
    checkpoint_path_weights = Path('./checkpoints/autoencoder/weights.ckpt')
    if not checkpoint_path_weights.exists():
        print("No autoencoder weights found!")
    else:
        autoencoder = AutoEncoder(nb_channels_in=len(ALL_INPUTS), nb_channels_out=OUTPUT_FEATURES,
                           time_steps=SEQUENCE_LENGTH, img_size=IMG_SIZE)
        autoencoder.load_weights(checkpoint_path_weights)

        print("Reducing data dimension")
        inputs = autoencoder.encoder.predict(inputs)
        INPUT_CHANNELS = AUTOENCODER_OUTPUT_FEATURES

No autoencoder weights found!


In [16]:
#import sys
#if 'gan.train' in sys.modules:
 #   del sys.modules['gan.train']
#from gan import train

In [17]:
from gan import train, metrics
from gan.models import make_generator, make_discriminator
generator = make_generator(image_size=IMG_SIZE, in_channels=INPUT_CHANNELS,
                           noise_channels=NOISE_CHANNELS, out_channels=OUT_CHANNELS,
                           n_timesteps=SEQUENCE_LENGTH)
generator.compile(train.generator_optimizer())
print(f"Generator: {generator.count_params():,} weights")

discriminator = make_discriminator(low_res_size=IMG_SIZE, high_res_size=IMG_SIZE, low_res_channels=INPUT_CHANNELS,
                                   high_res_channels=OUT_CHANNELS, n_timesteps=SEQUENCE_LENGTH)
discriminator.compile(train.discriminator_optimizer(), train.discriminator_loss)
print(f"Discriminator: {discriminator.count_params():,} weights")

Generator: 2,600,513 weights
Discriminator: 36,361,309 weights


In [18]:
from gan.ganbase import  GAN
from data.data_generator import FlexibleNoiseGenerator

noise_shape = (BATCH_SIZE, SEQUENCE_LENGTH, IMG_SIZE, IMG_SIZE, NOISE_CHANNELS)
gan = GAN(generator, discriminator, noise_generator=FlexibleNoiseGenerator(noise_shape, std=1))

print(f"Total: {gan.generator.count_params() + gan.discriminator.count_params():,} weights")

Total: 38,961,822 weights


In [19]:
gan.compile(generator_optimizer=train.generator_optimizer(),
            generator_metrics= [tf.keras.metrics.RootMeanSquaredError(), metrics.LogSpectralDistance(),
                                metrics.WeightedRMSEForExtremes()],
            discriminator_optimizer=train.discriminator_optimizer(),
                discriminator_loss=train.discriminator_loss,
           metrics = [metrics.discriminator_score_fake(), metrics.discriminator_score_real()])

In [20]:
checkpoint_path_weights = Path('./checkpoints/gan/weights-{epoch:02d}.ckpt')
checkpoint_path_weights.parent.mkdir(exist_ok=True, parents=True)
log_path = Path('./logs/gan')
if log_path.exists():
    log_path_str = str(log_path)
    !rm -rf {log_path_str}

In [21]:
%load_ext tensorboard
%tensorboard --logdir=logs/gan

In [22]:
import matplotlib.pyplot as plt
def show(images, dims=1, legends=None):
    fig, axes = plt.subplots(ncols= len(images), figsize=(10, 10))
    for ax, im in zip(axes, images):
        for i in range(dims):
            label = legends[i] if legends is not None else ''
            ax.imshow(im[0, :, :, i], cmap='jet')
            ax.set_title(label)
            ax.axis('off')
    fig.show()

In [23]:
class ShowCallback(tf.keras.callbacks.Callback):
    def __init__(self, dims):
        self.dims = dims
        
    def on_epoch_begin(self, epoch, logs):
        noise = FlexibleNoiseGenerator(noise_shape)
        show(self.model.generator([inputs[:self.dims], noise(self.dims)]))

In [24]:
callbacks = [
    cb.TensorBoard(log_path, write_images=True, update_freq='batch', profile_batch=(2, 4)),
    cb.ProgbarLogger('steps'),
    cb.TerminateOnNaN(),
    ShowCallback(10),
    cb.ModelCheckpoint(str(checkpoint_path_weights), monitor='root_mean_squared_error', mode='min',
                       save_best_only=False, save_weights_only=True),
]

In [ ]:
gan.fit(inputs, outputs, callbacks=callbacks, epochs=1, batch_size=BATCH_SIZE, validation_split=0.25)

<ipython-input-22-971d8efb5882>:10: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()


 1/47 [..............................] - ETA: 27:23 - d_loss: 0.0016 - g_loss: 1.0229e-05 - d_fake: 1.0229e-05 - d_real: 0.0016 - g_root_mean_squared_error: 2.6630 - g_lsd: 0.0000e+00 - g_extreme_rmse: 0.0000e+00

In [ ]:
i=0
noise = FlexibleNoiseGenerator(noise_shape)()
results = gan.generator.predict([inputs[i:i+BATCH_SIZE], noise])

In [ ]:
for epoch in [7,14,35,38,42,50]:
    gan.load_weights(f'checkpoints/gan/weights-{epoch:02d}.ckpt')
    noise = FlexibleNoiseGenerator(noise_shape)()
    #os.mkdir(f'plots/gan_pred/{epoch}')
    for i in range(0, BATCH_SIZE*6, BATCH_SIZE):
        results = gan.generator.predict([inputs[i:i+BATCH_SIZE], noise])
        j=0
        for inp, out, res in zip(inputs[i:i+BATCH_SIZE], outputs[i:i+BATCH_SIZE], results):
            fig = show([inp, out, res])
            #fig.savefig(f'plots/gan_pred/{epoch}/inp_{i}_batch_{j}.png')
            j+=1

In [ ]:
epoch=1

gan.load_weights(f'checkpoints/gan/weights-{epoch:02d}.ckpt')